In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el modelo y el tokenizador
modelo = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn', output_attentions=True)
tokenizador = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Texto de ejemplo
texto = (
    "La inteligencia artificial (IA) se refiere a la simulación de procesos de inteligencia humana "
    "por parte de máquinas, especialmente sistemas informáticos. Estos procesos incluyen el aprendizaje "
    "(la adquisición de información y reglas para el uso de la información), el razonamiento (uso de reglas "
    "para alcanzar conclusiones aproximadas o definitivas) y la autocorrección. Las aplicaciones particulares "
    "de la IA incluyen sistemas expertos, reconocimiento de voz y visión artificial."
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Tokenizar el texto de entrada
entradas = tokenizador(texto, max_length=3000, return_tensors='pt', truncation=True)

# Generar el resumen
salida = modelo.generate(entradas['input_ids'], num_beams=4, max_length=400, early_stopping=True)

# Extraer el resumen
resumen_ids = salida[0]

# Decodificar el resumen generado
resumen = tokenizador.decode(resumen_ids, skip_special_tokens=True)
print("Resumen:")
print(resumen)

BartModel is using BartSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Resumen:
La inteligencia artificial (IA) se refiere a la simulación de procesos de inteligence humana por parte de máquinas. Las aplicaciones particulares de la IA incluyen sistemas expertos, reconocimiento de voz y visión artificial.


In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Cargar modelo y tokenizador
modelo = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn', output_attentions=True)
tokenizador = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Datos de prueba
articulos = [
    "Este es un ejemplo de artículo para demostrar la capacidad del modelo de generar resúmenes.",
    "Otro ejemplo de artículo que será resumido por el modelo BART para evaluar su rendimiento."
]

resumenes_referencia = [
    "Ejemplo de artículo para demostrar el modelo de generar resúmenes.",
    "Ejemplo de artículo resumido por el modelo BART para evaluar su rendimiento."
]

# Función para generar resúmenes
def generar_resumenes(articulos, modelo, tokenizador):
    resumenes_generados = []
    for articulo in articulos:
        entradas = tokenizador(articulo, max_length=1024, return_tensors='pt', truncation=True)
        salida = modelo.generate(entradas['input_ids'], num_beams=4, max_length=150, early_stopping=True)
        resumen_generado = tokenizador.decode(salida[0], skip_special_tokens=True)
        resumenes_generados.append(resumen_generado)
    return resumenes_generados

# Función para calcular ROUGE-N (unigramas y bigramas)
def calcular_rouge_n(predicciones, referencias, n=1):
    def ngrams(texto, n):
        tokens = texto.split()
        return set(zip(*[tokens[i:] for i in range(n)]))

    puntajes = []
    for prediccion, referencia in zip(predicciones, referencias):
        pred_ngrams = ngrams(prediccion, n)
        ref_ngrams = ngrams(referencia, n)
        interseccion_ngrams = pred_ngrams.intersection(ref_ngrams)
        if len(pred_ngrams) == 0 or len(ref_ngrams) == 0:
            puntaje = 0.0
        else:
            puntaje = 2 * len(interseccion_ngrams) / (len(pred_ngrams) + len(ref_ngrams))
        puntajes.append(puntaje)
    return sum(puntajes) / len(puntajes)

# Función para calcular ROUGE-L
def calcular_rouge_l(predicciones, referencias):
    def lcs(X, Y):
        m = len(X)
        n = len(Y)
        L = [[0] * (n + 1) for i in range(m + 1)]
        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0 or j == 0:
                    L[i][j] = 0
                elif X[i-1] == Y[j-1]:
                    L[i][j] = L[i-1][j-1] + 1
                else:
                    L[i][j] = max(L[i-1][j], L[i][j-1])
        return L[m][n]

    puntajes = []
    for prediccion, referencia in zip(predicciones, referencias):
        lcs_length = lcs(prediccion.split(), referencia.split())
        if len(prediccion.split()) == 0 or len(referencia.split()) == 0:
            puntaje = 0.0
        else:
            puntaje = 2 * lcs_length / (len(prediccion.split()) + len(referencia.split()))
        puntajes.append(puntaje)
    return sum(puntajes) / len(puntajes)

# Generar resúmenes
resumenes_generados = generar_resumenes(articulos, modelo, tokenizador)

# Calcular métricas ROUGE
rouge_1 = calcular_rouge_n(resumenes_generados, resumenes_referencia, n=1)
rouge_2 = calcular_rouge_n(resumenes_generados, resumenes_referencia, n=2)
rouge_l = calcular_rouge_l(resumenes_generados, resumenes_referencia)

print(f"ROUGE-1: {rouge_1:.4f}")
print(f"ROUGE-2: {rouge_2:.4f}")
print(f"ROUGE-L: {rouge_l:.4f}")

ROUGE-1: 0.5559
ROUGE-2: 0.3963
ROUGE-L: 0.4645


In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Cargar modelo y tokenizador
modelo = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn', output_attentions=True)
tokenizador = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Datos de prueba
articulos = [
    "Este es un ejemplo de artículo para demostrar la capacidad del modelo de generar resúmenes.",
    "Otro ejemplo de artículo que será resumido por el modelo BART para evaluar su rendimiento."
]

resumenes_referencia = [
    "Ejemplo de artículo para demostrar el modelo de generar resúmenes.",
    "Ejemplo de artículo resumido por el modelo BART para evaluar su rendimiento."
]

# Función para generar resúmenes y capturar atención
def generar_resumenes_con_atencion(articulos, modelo, tokenizador):
    resumenes_generados = []
    atenciones = []
    for articulo in articulos:
        entradas = tokenizador(articulo, max_length=1024, return_tensors='pt', truncation=True)
        salida = modelo.generate(entradas['input_ids'], num_beams=4, max_length=150, early_stopping=True, output_attentions=True)
        resumen_generado = tokenizador.decode(salida[0], skip_special_tokens=True)
        atencion = salida[-1]
        resumenes_generados.append(resumen_generado)
        atenciones.append(atencion)
    return resumenes_generados, atenciones

# Generar resúmenes y capturar atención
resumenes_generados, atenciones = generar_resumenes_con_atencion(articulos, modelo, tokenizador)

# Imprimir resúmenes generados y matrices de atención
for idx, (resumen_generado, atencion) in enumerate(zip(resumenes_generados, atenciones)):
    print(f"Artículo de entrada {idx + 1}:")
    print(articulos[idx])
    print("\nResumen Generado:")
    print(resumen_generado)
    print("\nMatriz de Atención:")
    print(atencion)
    print("="*50)



Artículo de entrada 1:
Este es un ejemplo de artículo para demostrar la capacidad del modelo de generar resúmenes.

Resumen Generado:
Este es un ejemplo de artículo para demostrar la capacidad del modelo de generar resúmenes, según el projector. El projecto fue diseñado para un evento de eventos de la UNICEF.

Matriz de Atención:
tensor([    2,     0,   717,  6526,  2714,   542,   364,   267, 33037,   139,
          263,  1808,  1977, 13300,   139,  3840,  4410,  2603, 17254,   897,
        30862,  8843,  2424,  1421,   139,   263, 20181,   271,  5032,  5874,
         2262,   293,     6,   842,   571,  5874,   282,  1615,   695,   368,
            4,  1448,   695,   139, 13081, 40177,  6303,  2102,  3840,   542,
          515,   139,   263,   515,   366,   263,   897,  2604,  9292,   597,
            4,     2])
Artículo de entrada 2:
Otro ejemplo de artículo que será resumido por el modelo BART para evaluar su rendimiento.

Resumen Generado:
Otro ejemplo de artículo que será resumido p

In [ ]:
import numpy as np

# Función para calcular la coherencia de atención
def calcular_coherencia_atencion(atenciones):
    coherencias = []
    for atencion in atenciones:
        # Convertir la lista de matrices de atención en un array de NumPy
        atencion_np = np.array(atencion)
        # Calcular la desviación estándar de las ponderaciones de atención para cada artículo
        desviacion_estandar = np.std(atencion_np, axis=-1).mean()
        coherencias.append(desviacion_estandar)
    return coherencias

# Calcular la coherencia de atención para los diferentes mecanismos
coherencias = calcular_coherencia_atencion(atenciones)

# Imprimir la coherencia de atención para cada mecanismo
for idx, coherencia in enumerate(coherencias):
    print(f"Mecanismo de atención {idx + 1}: Coherencia = {coherencia:.4f}")


Mecanismo de atención 1: Coherencia = 8077.2371
Mecanismo de atención 2: Coherencia = 10835.6303
